# Basic application

This notebook shows you how to use the *basic* Vitis network example that can be generated from this repository.
The design provides network connectivity using User Datagram Protocol (UDP) as the transport protocol.

The assumptions made is this notebook are:
1. You have an Alveo card configured with one of the supported shells
1. You have generated the xclbin file for the *basic* example
1. You have a 100 GbE capable NIC
1. Both the Alveo card and the 100 GbE capable NIC are in the same host
1. Alveo card is connected to the NIC, either directly or with a any network equipment, using any of the interfaces.

Let's have a look at the *basic* design.

There are 4 Kernels:
* CMAC: provides the translation between physical signals to AXI4-Stream interface
* Network layer: provides a bridge between raw Ethernet packets and the application using UDP as transport layer
    * ARP provides translation between MAC and IP addresses
    * ICMP provides ping capabilities
    * The UDP module has a 16-entry table with socket information that needs to be filled in before running
* krnl_mm2s: reads data from memory and packetize it setting tdest and tlast appropriately
* krnl_s2mm: read data from the stream and copy it to memory

![](../img/udp_network_basic.png)

In the followings cells we will:
* Import the necessary pynq and python packages, define current device and xclbin file
* Explore kernels in the design
* Check physical link
* Change Alveo card IP address and ping it
* Configure socket table and populate it to the UDP module
* Create UDP socket in the host
* Allocate Alveo buffers
* Move data from the HOST through the network (NIC) to the Alveo card
* Move data from the HOST through the Alveo card to the network (NIC)
* Free resources

In [20]:
#Uncomment the next line and run to reset the FPGA if it is not taking programming or otherwise misbehaving
#!xbutil reset --device 0000:02:00.1 --force

## Import packages and program FPGA
In this section we need to import the `pynq` and python packages that will be used in the rest of this notebook. We also import the `vnx_utils.py` file with helper functions to set up the vnx examples.

In [21]:
import pynq
import numpy as np
from _thread import *
import threading 
import socket
from vnx_utils import *

In [22]:
# Resource report summary (if available)
import os

rpt = '/home/m2_2/shared/dat480-final/project_split.intf0.xilinx_u55c_gen3x16_xdma_3_202210_1/reports/link/system_estimate_vnx_project_split_if0.xtxt'
if os.path.exists(rpt):
    print("Resource report:", rpt)
    lines = open(rpt, 'r').read().splitlines()
    start = None
    for i, line in enumerate(lines):
        if line.strip() == 'Area Information':
            start = i
            break
    if start is None:
        print("Area Information section not found")
    else:
        first_sep = None
        second_sep = None
        for i in range(start + 1, len(lines)):
            if lines[i].startswith('----'):
                first_sep = i
                break
        if first_sep is not None:
            for i in range(first_sep + 1, len(lines)):
                if lines[i].startswith('----'):
                    second_sep = i
                    break
        if first_sep is None or second_sep is None:
            print("Area Information table not found")
        else:
            table = lines[start:second_sep + 1]
            for line in table:
                print(line)

            # Parse usage and compute percentage within this table
            data_rows = [l for l in table if l.startswith('krnl_proj_split_0')]
            totals = {'FF': 0, 'LUT': 0, 'DSP': 0, 'BRAM': 0, 'URAM': 0}
            parsed = []
            for row in data_rows:
                parts = row.split()
                if len(parts) >= 9:
                    ff, lut, dsp, bram, uram = map(int, parts[-5:])
                    totals['FF'] += ff
                    totals['LUT'] += lut
                    totals['DSP'] += dsp
                    totals['BRAM'] += bram
                    totals['URAM'] += uram
                    parsed.append((parts[2], ff, lut, dsp, bram, uram))

            if sum(totals.values()) > 0:
                print("Resource share within krnl_proj_split table:")
                for name, ff, lut, dsp, bram, uram in parsed:
                    def pct(v, t):
                        return 0.0 if t == 0 else (100.0 * v / t)
                    print("{}: FF {:.2f}%, LUT {:.2f}%, DSP {:.2f}%, BRAM {:.2f}%, URAM {:.2f}%".format(
                        name, pct(ff, totals['FF']), pct(lut, totals['LUT']), pct(dsp, totals['DSP']),
                        pct(bram, totals['BRAM']), pct(uram, totals['URAM'])
                    ))
            else:
                print("No data rows found for krnl_proj_split_0")
else:
    print("Resource report not found:", rpt)

Resource report: /home/m2_2/shared/dat480-final/project_split.intf0.xilinx_u55c_gen3x16_xdma_3_202210_1/reports/link/system_estimate_vnx_project_split_if0.xtxt
Area Information
Compute Unit       Kernel Name      Module Name                                      FF    LUT    DSP  BRAM  URAM
-----------------  ---------------  -----------------------------------------------  ----  -----  ---  ----  ----
krnl_proj_split_0  krnl_proj_split  input_split_Pipeline_VITIS_LOOP_39_2             148   2668   0    0     0
krnl_proj_split_0  krnl_proj_split  input_split                                      877   2812   0    0     0
krnl_proj_split_0  krnl_proj_split  matcher_engine_1720_16_Pipeline_init_state       13    63     0    0     0
krnl_proj_split_0  krnl_proj_split  matcher_engine_1720_16_Pipeline_pattern_loop     118   22295  0    19    0
krnl_proj_split_0  krnl_proj_split  matcher_engine_1720_16_Pipeline_reset_state      13    63     0    0     0
krnl_proj_split_0  krnl_proj_split  matc

We also need to define the current device, only if there is more than one Alveo card on the host. First let's check how many devices are available.

In [23]:
for i in range(len(pynq.Device.devices)):
    print("{}) {}".format(i, pynq.Device.devices[i].name))

0) xilinx_u55c_gen3x16_xdma_base_3


* If there are more than one Alveo card available, you should pass the `device` argument to the `pynq.Overlay` class.
* The xclbin variable should point to the `xclbin` file for the *basic* example

In [24]:
currentDevice = pynq.Device.devices[0]
xclbin = '/home/m2_2/shared/dat480-final/project_split.intf0.xilinx_u55c_gen3x16_xdma_3_202210_1/vnx_project_split_if0.xclbin'
try:
    ol = pynq.Overlay(xclbin, device=currentDevice)
except RuntimeError as exc:
    print("Overlay download failed, try reuse existing xclbin:", exc)
    ol = pynq.Overlay(xclbin, device=currentDevice, download=False)

## Explore kernels in the design
This design was built for both interfaces. Therefore, we will see each kernel repeated twice. One for the interface 0 and another for the interface 1

In [25]:
ol.ip_dict

{'cmac_0': {'phys_addr': 12582912,
  'addr_range': 8192,
  'type': 'xilinx.com:kernel:cmac_0:1.0',
  'hw_control_protocol': 'ap_ctrl_none',
  'fullpath': 'cmac_0',
  'registers': {'gt_reset_reg': {'address_offset': 0,
    'access': 'read-write;',
    'size': 32,
    'host_size': 4,
    'description': 'OpenCL Argument Register',
    'type': 'uint',
    'id': 0},
   'reset_reg': {'address_offset': 4,
    'access': 'read-write;',
    'size': 32,
    'host_size': 4,
    'description': 'OpenCL Argument Register',
    'type': 'uint',
    'id': 1},
   'mode': {'address_offset': 8,
    'access': 'read-write;',
    'size': 32,
    'host_size': 4,
    'description': 'OpenCL Argument Register',
    'type': 'uint',
    'id': 2},
   'conf_tx': {'address_offset': 12,
    'access': 'read-write;',
    'size': 32,
    'host_size': 4,
    'description': 'OpenCL Argument Register',
    'type': 'uint',
    'id': 3},
   'conf_rx': {'address_offset': 20,
    'access': 'read-write;',
    'size': 32,
    'hos

## Check physical link
After the dynamic region of the Alveo card is programmed with the basic example we can start interacting with the design.

Let's check if the Alveo card has detected link with the network equipment. To do so, we will use one of the helper functions `link_status`

In [26]:
print("Link interface 0 {}".format(ol.cmac_0.link_status()))

Link interface 0 {'cmac_link': True}


## Change Alveo card IP address and ping it
By defaul the Alveo IP address is `192.168.0.5` and MAC address is `00:0A:35:02:9D:E5`. Let's change it to `192.168.100.2`, in this example we are using interface 0

After the IP address is changed, we can ping the Alveo card. The first attempts will fail but the remaining should work.

**Make sure you have configured the IP address of the 100 GbE capable NIC to be in the same subnetwork as the Alveo card**

In [27]:
alveo_ipaddr = '192.168.100.2'
print(ol.networklayer_0.set_ip_address(alveo_ipaddr, debug=True))

{'HWaddr': '00:0a:35:02:9d:02', 'inet addr': '192.168.100.2', 'gateway addr': '192.168.100.1', 'Mask': '255.255.255.0'}


* Check 100 GbE capable NIC configuration

In [28]:
!ip addr show ens1f0np0

3: ens1f0np0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 0c:42:a1:ef:fd:96 brd ff:ff:ff:ff:ff:ff
    altname enp1s0f0np0
    inet 192.168.100.1/30 scope global ens1f0np0
       valid_lft forever preferred_lft forever
    inet6 fe80::e42:a1ff:feef:fd96/64 scope link 
       valid_lft forever preferred_lft forever


* Run ping to get ARP set up
Some of the attempts will fail but the remaining should work.

In [29]:
!ping -c 5 $alveo_ipaddr -I ens1f0np0

PING 192.168.100.2 (192.168.100.2) from 192.168.100.1 ens1f0np0: 56(84) bytes of data.


From 192.168.100.1 icmp_seq=1 Destination Host Unreachable
From 192.168.100.1 icmp_seq=2 Destination Host Unreachable
From 192.168.100.1 icmp_seq=3 Destination Host Unreachable
From 192.168.100.1 icmp_seq=4 Destination Host Unreachable
From 192.168.100.1 icmp_seq=5 Destination Host Unreachable

--- 192.168.100.2 ping statistics ---
5 packets transmitted, 0 received, +5 errors, 100% packet loss, time 4073ms
pipe 3


## Configure socket table and populate it to the UDP module

In this section we will configure the socket table in software and populate it to the UDP module in the Alveo card. 
1. Define a couple of connections 
1. The socket table is populated to the UDP module in the Alveo card using the `.populate_socket_table()` helper function

In [30]:
sw_ip = '192.168.100.1'
host_port = 38746
fpga_port = 62781
ol.networklayer_0.sockets[0] = (sw_ip, host_port, fpga_port, True)

numSocketsHW = int(ol.networklayer_0.register_map.udp_number_sockets)
if numSocketsHW == 0:
    print("UDP sockets in hardware is 0; check link/overlay and retry.")
else:
    ol.networklayer_0.populate_socket_table(debug=True)

# dump_net_state(ol)

## Create UDP socket in the host
In this part we will open an UDP socket in the host (software) to be able to communicate with the Alveo card through the network.

We will use the python socket API to do so, the socket will be binded to the port `38746`

In [31]:
SW_PORT = ol.networklayer_0.sockets[0]['theirPort']
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # UDP
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF, 4 * 1024 * 1024)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_RCVBUF, 4 * 1024 * 1024)
sock.bind(('', SW_PORT))
sock.settimeout(2.0)

In [32]:
import time


def drain_socket(sock, max_packets=1024, timeout_s=0.001):
    orig_timeout = sock.gettimeout()
    sock.settimeout(timeout_s)
    count = 0
    while count < max_packets:
        try:
            sock.recvfrom(2048)
            count += 1
        except socket.timeout:
            break
    sock.settimeout(orig_timeout)
    return count


def recv_matches(sock, max_packets=64, per_packet_bytes=4096,
                 idle_timeout_s=0.02, total_timeout_s=0.2):
    orig_timeout = sock.gettimeout()
    sock.settimeout(idle_timeout_s)
    ids = []
    packets = 0
    addr = None
    start = time.monotonic()
    while packets < max_packets and (time.monotonic() - start) < total_timeout_s:
        try:
            data, addr = sock.recvfrom(per_packet_bytes)
            packets += 1
            for i in range(0, len(data), 4):
                if i + 4 <= len(data):
                    ids.append(int.from_bytes(data[i:i + 4], "little"))
        except socket.timeout:
            if ids:
                break
    sock.settimeout(orig_timeout)
    return ids, packets, addr


def build_payload():
    parts = [
        b"USER admin logged in; ",
        b"FTP Port open; ",
        b"backdoor detected; ",
        b"r00t access granted"
    ]
    return b"".join(parts)


def dump_net_state(ol):
    print("Link status:", ol.cmac_0.link_status())
    print("Network info:", ol.networklayer_0.get_network_info())
    print("ARP table:", ol.networklayer_0.get_arp_table(num_entries=16))


def reset_udp_path(ol, sock, sw_ip, host_port, fpga_port):
    try:
        sock.close()
    except Exception:
        pass

    ol.networklayer_0.invalidate_socket_table()
    ol.networklayer_0.invalidate_arp_table()
    ol.networklayer_0.sockets[0] = (sw_ip, host_port, fpga_port, True)
    numSocketsHW = int(ol.networklayer_0.register_map.udp_number_sockets)
    if numSocketsHW == 0:
        print("UDP sockets in hardware is 0; check link/overlay and retry.")
    else:
        ol.networklayer_0.populate_socket_table()
        ol.networklayer_0.arp_discovery()

    new_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    new_sock.setsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF, 4 * 1024 * 1024)
    new_sock.setsockopt(socket.SOL_SOCKET, socket.SO_RCVBUF, 4 * 1024 * 1024)
    new_sock.bind(('', host_port))
    new_sock.settimeout(2.0)
    return new_sock


def full_soft_reset(ol, sock, sw_ip, host_port, fpga_port, alveo_ipaddr):
    try:
        sock.close()
    except Exception:
        pass

    try:
        ol.networklayer_0.set_ip_address(alveo_ipaddr, debug=False)
    except Exception:
        pass

    ol.networklayer_0.invalidate_socket_table()
    ol.networklayer_0.invalidate_arp_table()
    ol.networklayer_0.sockets[0] = (sw_ip, host_port, fpga_port, True)
    numSocketsHW = int(ol.networklayer_0.register_map.udp_number_sockets)
    if numSocketsHW == 0:
        print("UDP sockets in hardware is 0; check link/overlay and retry.")
    else:
        ol.networklayer_0.populate_socket_table()
        ol.networklayer_0.arp_discovery()

    try:
        krnl.register_map.CTRL.AP_START = 1
    except Exception:
        pass

    time.sleep(0.05)

    new_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    new_sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    new_sock.setsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF, 4 * 1024 * 1024)
    new_sock.setsockopt(socket.SOL_SOCKET, socket.SO_RCVBUF, 4 * 1024 * 1024)
    new_sock.bind(('', host_port))
    new_sock.settimeout(2.0)
    return new_sock


def reinit_overlay(xclbin, device, sw_ip, host_port, fpga_port, alveo_ipaddr, old_sock=None):
    try:
        if old_sock is not None:
            old_sock.close()
    except Exception:
        pass

    try:
        pynq.Overlay.free(ol)
    except Exception:
        pass

    new_ol = pynq.Overlay(xclbin, device=device)
    new_ol.networklayer_0.set_ip_address(alveo_ipaddr, debug=False)
    new_ol.networklayer_0.sockets[0] = (sw_ip, host_port, fpga_port, True)
    numSocketsHW = int(new_ol.networklayer_0.register_map.udp_number_sockets)
    if numSocketsHW == 0:
        print("UDP sockets in hardware is 0; check link/overlay and retry.")
    else:
        new_ol.networklayer_0.populate_socket_table()
        new_ol.networklayer_0.arp_discovery()

    new_krnl = new_ol.krnl_proj_split_0
    new_krnl.register_map.CTRL.AUTO_RESTART = 1
    new_krnl.register_map.CTRL.AP_START = 1

    new_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    new_sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    new_sock.setsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF, 4 * 1024 * 1024)
    new_sock.setsockopt(socket.SOL_SOCKET, socket.SO_RCVBUF, 4 * 1024 * 1024)
    new_sock.bind(('', host_port))
    new_sock.settimeout(2.0)
    return new_ol, new_krnl, new_sock


def print_net_deltas(before_nl, after_nl, before_cmac, after_cmac):
    def delta(path, key):
        return after_nl[path][key]["packets"] - before_nl[path][key]["packets"]

    print("NL delta rx eth packets:", delta("rx_path", "ethernet"))
    print("NL delta rx arp packets:", delta("rx_path", "arp"))
    print("NL delta rx icmp packets:", delta("rx_path", "icmp"))
    print("NL delta rx udp packets:", delta("rx_path", "udp"))
    print("NL delta rx app packets:", delta("rx_path", "app"))
    print("NL delta tx eth packets:", delta("tx_path", "ethernet"))
    print("NL delta tx arp packets:", delta("tx_path", "arp"))
    print("NL delta tx icmp packets:", delta("tx_path", "icmp"))
    print("NL delta tx udp packets:", delta("tx_path", "udp"))
    print("NL delta tx app packets:", delta("tx_path", "app"))

    tx_pkts = after_cmac["tx"]["packets"] - before_cmac["tx"]["packets"]
    rx_pkts = after_cmac["rx"]["packets"] - before_cmac["rx"]["packets"]
    print("CMAC delta tx packets:", tx_pkts)
    print("CMAC delta rx packets:", rx_pkts)

## Allocate Alveo buffers

We need to allocate the buffers on the global memory of the Alveo card to be able to pull and push data from them. We also will initialize the the sending buffer, `mm2s_buf`, with random data. This random data will be sent later to the network.

1. Define alias for the application kernlels (`mm2s` and `s2mm`)
1. Define size and shape of the buffers
1. Allocate the buffers
1. Initialize sending buffer with random data

In [33]:
krnl = ol.krnl_proj_split_0
# The design has no active memory banks; start via control register
krnl.register_map.CTRL.AUTO_RESTART = 1
krnl.register_map.CTRL.AP_START = 1


## Move data from the HOST through the network (NIC) to the Alveo card

In this section we will write data to the socket, which will send such data from the host to the Alveo card using the network.

Start streaming to memory mapped kernel, we need to specify how much data to expect from the network.

In [34]:
payload = build_payload()

# Ensure kernel is running
try:
    krnl.register_map.CTRL.AP_START = 1
except Exception:
    pass

# Clear counters and drain host socket
try:
    ol.networklayer_0.reset_debug_stats()
except Exception:
    pass

before_nl = ol.networklayer_0.get_debug_stats
before_cmac = ol.cmac_0.get_stats()

_ = drain_socket(sock, max_packets=512, timeout_s=0.001)

# Refresh ARP on FPGA side before request
ol.networklayer_0.arp_discovery()

sock.sendto(payload, (alveo_ipaddr, fpga_port))

ids, pkt_count, addr = recv_matches(sock, max_packets=32,
                                   per_packet_bytes=4096,
                                   idle_timeout_s=0.02,
                                   total_timeout_s=0.3)

after_nl = ol.networklayer_0.get_debug_stats
after_cmac = ol.cmac_0.get_stats()
print_net_deltas(before_nl, after_nl, before_cmac, after_cmac)

if pkt_count == 0:
    print("No response received (timeout).")
else:
    unique_ids = sorted(set(ids))
    print("Received packets={}, ids_count={}, unique_ids={}".format(
        pkt_count, len(ids), len(unique_ids)))
    print("Match IDs (first 32):", unique_ids[:32])

NL delta rx eth packets: 0
NL delta rx arp packets: 0
NL delta rx icmp packets: 0
NL delta rx udp packets: 0
NL delta rx app packets: 0
NL delta tx eth packets: 256
NL delta tx arp packets: 256
NL delta tx icmp packets: 0
NL delta tx udp packets: 0
NL delta tx app packets: 0
CMAC delta tx packets: -256
CMAC delta rx packets: -9
No response received (timeout).


In [35]:
# Throughput test: send many UDP packets and measure Tx rate
# Drain and report responses during the loop to avoid backpressure and show live results.
import time

num_pkts = 100
pkt_size = 1472
payload_thr = b'A' * pkt_size

before_nl = ol.networklayer_0.get_debug_stats
before_cmac = ol.cmac_0.get_stats()

_ = drain_socket(sock, max_packets=512, timeout_s=0.001)

start_ts = time.perf_counter()
received_ids = []
last_report = time.perf_counter()
for i in range(num_pkts):
    sock.sendto(payload_thr, (alveo_ipaddr, fpga_port))
    if (i + 1) % 16 == 0:
        ids, pkt_count, _ = recv_matches(
            sock, max_packets=8, per_packet_bytes=4096,
            idle_timeout_s=0.002, total_timeout_s=0.01
        )
        if pkt_count:
            received_ids.extend(ids)
        now = time.perf_counter()
        if now - last_report >= 0.2:
            uniq = len(set(received_ids))
            print("Progress sent={}/{} received_packets={} unique_ids={}".format(
                i + 1, num_pkts, pkt_count, uniq))
            last_report = now
end_ts = time.perf_counter()

# Final drain and report
ids, pkt_count, _ = recv_matches(
    sock, max_packets=64, per_packet_bytes=4096,
    idle_timeout_s=0.01, total_timeout_s=0.05
)
if pkt_count:
    received_ids.extend(ids)

uniq = len(set(received_ids))
print("Final received_packets={} unique_ids={}".format(pkt_count, uniq))

after_nl = ol.networklayer_0.get_debug_stats
after_cmac = ol.cmac_0.get_stats()
print_net_deltas(before_nl, after_nl, before_cmac, after_cmac)

elapsed = end_ts - start_ts
bytes_sent = num_pkts * pkt_size
if elapsed > 0:
    gbps = (bytes_sent * 8.0) / (elapsed * 1e9)
else:
    gbps = 0.0

print("Sent bytes={}, elapsed_s={:.6f}, tx_gbps={:.3f}".format(bytes_sent, elapsed, gbps))

Final received_packets=0 unique_ids=0
NL delta rx eth packets: 0
NL delta rx arp packets: 0
NL delta rx icmp packets: 0
NL delta rx udp packets: 0
NL delta rx app packets: 0
NL delta tx eth packets: 0
NL delta tx arp packets: 0
NL delta tx icmp packets: 0
NL delta tx udp packets: 0
NL delta tx app packets: 0
CMAC delta tx packets: 0
CMAC delta rx packets: 100
Sent bytes=147200, elapsed_s=0.063369, tx_gbps=0.019


In [36]:
# If timeouts persist, try the soft reset below and retry.
sock = full_soft_reset(ol, sock, sw_ip, host_port, fpga_port, alveo_ipaddr)
# If still no UDP rx, reinitialize the overlay (no sudo required).
ol, krnl, sock = reinit_overlay(
    xclbin, currentDevice, sw_ip, host_port, fpga_port, alveo_ipaddr, old_sock=sock
)

Initialize a buffer with random data, define the packet size and compute how many packets we need to send to transmit the whole buffer to the network. Write the data from the buffer into the socket in `BYTES_PER_PACKET` chunks.

In [37]:
sock.close()
pynq.Overlay.free(ol)

* Wait for the s2mm kernel to receive all the data
* Move data from global memory to HOST memory
* Compare what was sent against what was received and print out result

In [38]:
if 'ol' in globals():
    dump_net_state(ol)
else:
    print("Overlay not initialized")

OSError: [Errno 19] No such device

## Move data from the HOST through the Alveo card to the network (NIC)

We need to create a new thread to read data from the socket, this mainly because there will be multiple packets. This is done in the `socket_receive_threaded` function

* Copy random data to Alveo global memory
* Acquire thread 
* Launch threaded function
* Start kernel memory mapped to stream kernel, this will start sending UDP packets from the Alveo card to the NIC
* Threaded function will start receiving data and it will print out result once ALL data was collected

## Free resources
Delete buffers and free Alveo card

------------------------------------------
Copyright (c) 2020-2021, Xilinx, Inc.